# **Lactate Threshold Calculator**

**Setup**

In [1]:
# INPUTS ****************

time_intervals = ["8-9", "13-14", "18-19", "23-24", "28-29", "33-34", "38-39", "43-44"] # Format is "min-min"
stages = [1, 2, 3, 4, 5, 6, 7, 8]

print_HR_graph = False
print_all_HR = False

# ***********************


# === OVERALL CODE FLOW:

#1 install libraries
#2 find activity id
#3 retrieve HR data from activity
#4 calculate 3rd quartiles for each time interval in activity
#5 manually determine the AeT and AnT stages via lactate values
#6 calculate HR best fit line and calculate AeT and AnT HR

**Installations**

In [2]:
%pip install -q garminconnect
%pip install -q ipywidgets
%pip install -q numpy
%pip install -q matplotlib
%pip install -q scipy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


**Print Activity IDs on Date**

In [ ]:
# INPUTS ****************

date = "2024-07-02"  # Format is "YYYY-MM-DD"

# Replace with your Garmin Connect credentials
username = ""
password = ""

# ***********************



from garminconnect import Garmin
from datetime import datetime

# Login to Garmin Connect
client = Garmin(username, password)
client.login()

# Function to get all activities for a specific date
def get_activities_for_date(client, date):
    """
    Retrieves all activities for a specific date.

    Parameters:
        client (Garmin): The authenticated Garmin client.
        date (str): The date in "YYYY-MM-DD" format.

    Returns:
        list: List of activities on the specified date.
    """
    try:
        # Fetch activities on the specified date
        activities = client.get_activities_by_date(date, date)

        if activities:
            print(f"Activities on {date}:")
            for activity in activities:
                print(f"\n\nActivity ID: {activity['activityId']}")
                print(f"Start Time: {activity['startTimeLocal']}")
                print(f"Activity Type: {activity['activityType']['typeKey']}")

                # Convert distance from meters to miles
                distance_miles = activity['distance'] / 1609.34

                # Calculate hours, minutes, and seconds from duration in seconds
                total_seconds = activity['duration']
                hours = total_seconds // 3600
                minutes = (total_seconds % 3600) // 60
                seconds = total_seconds % 60

                # Convert average speed from m/s to mph
                average_speed_mph = activity['averageSpeed'] * 2.23694

                print(f"Distance: {distance_miles:.2f} miles")
                print(f"Duration: {int(hours):02}:{int(minutes):02}:{int(seconds):02}")
                print(f"Average Speed: {average_speed_mph:.2f} mph")
                print(f"Calories: {activity['calories']} kcal")

        else:
            print(f"No activities found on {date}.")

    except Exception as e:
        print(f"An error occurred while retrieving activities: {e}")

# Call the function to retrieve and print activities for the specified date
get_activities_for_date(client, date)

Activities on 2024-07-02:


Activity ID: 16215601367
Start Time: 2024-07-02 20:08:18
Activity Type: treadmill_running
Distance: 7.02 miles
Duration: 01:01:00
Average Speed: 6.90 mph
Calories: 648.0 kcal


Activity ID: 16213792231
Start Time: 2024-07-02 16:32:40
Activity Type: running
Distance: 0.38 miles
Duration: 00:50:03
Average Speed: 0.46 mph
Calories: 539.0 kcal


Activity ID: 16212503162
Start Time: 2024-07-02 14:20:36
Activity Type: running
Distance: 0.42 miles
Duration: 00:51:55
Average Speed: 0.48 mph
Calories: 553.0 kcal


Activity ID: 16210113348
Start Time: 2024-07-02 12:10:35
Activity Type: running
Distance: 0.60 miles
Duration: 00:44:51
Average Speed: 0.80 mph
Calories: 514.0 kcal


Activity ID: 16207956684
Start Time: 2024-07-02 10:56:32
Activity Type: running
Distance: 1.08 miles
Duration: 00:10:11
Average Speed: 6.37 mph
Calories: 103.0 kcal


Activity ID: 16205675388
Start Time: 2024-07-02 08:05:38
Activity Type: running
Distance: 5.50 miles
Duration: 00:41:57
Average

**Retrieve HR Data**

In [5]:
# INPUTS ****************

activity_id = "16210113348"

# ***********************



import xml.etree.ElementTree as ET
import datetime

# Initialize list to hold all heart rates from all activities
all_heart_rates = []
zone_counts = {}

try:
    # Download and process each activity file
    from pathlib import Path

    # Save file in a local folder next to your notebook (or anywhere you choose)
    output_dir = Path.cwd() / "garmin_exports"
    output_dir.mkdir(parents=True, exist_ok=True)

    activity_data = client.download_activity(activity_id)
    file_name = output_dir / f"{activity_id}.tcx"

    with open(file_name, "wb") as file:
        file.write(activity_data)

    print(f"Saved TCX file to: {file_name}")
    
    # Parse and extract data from the .tcx file
    tree = ET.parse(file_name)
    root = tree.getroot()
    namespace = {
        'ns': 'http://www.garmin.com/xmlschemas/TrainingCenterDatabase/v2',
        'ns3': 'http://www.garmin.com/xmlschemas/ActivityExtension/v2'
    }

    # Loop through the trackpoints and extract heart rate data
    for trackpoint in root.findall('.//ns:Trackpoint', namespace):
        heart_rate_elem = trackpoint.find('.//ns:HeartRateBpm/ns:Value', namespace)
        if heart_rate_elem is not None:
            heart_rate = int(heart_rate_elem.text)
            all_heart_rates.append(heart_rate)

except Exception as e:
    print(f"Error processing activity {activity_id}: {e}")

print("All heart rates:")
print(all_heart_rates)

Saved TCX file to: /Users/dashiellcoyier/Desktop/lactate testing/garmin_exports/16210113348.tcx
All heart rates:
[102, 102, 102, 102, 102, 102, 102, 103, 103, 103, 103, 103, 103, 104, 104, 104, 106, 107, 107, 107, 108, 108, 110, 112, 114, 115, 117, 117, 118, 119, 120, 120, 120, 120, 121, 122, 122, 123, 123, 123, 123, 123, 123, 122, 123, 123, 123, 123, 122, 122, 123, 123, 124, 123, 123, 124, 124, 124, 124, 122, 121, 121, 121, 121, 122, 122, 123, 123, 123, 123, 123, 123, 123, 123, 123, 123, 123, 123, 124, 123, 123, 123, 123, 122, 122, 121, 121, 121, 120, 120, 120, 119, 119, 119, 119, 119, 120, 120, 120, 121, 121, 121, 121, 121, 122, 122, 122, 122, 122, 121, 121, 121, 121, 121, 121, 121, 121, 121, 121, 121, 121, 120, 121, 120, 121, 121, 121, 121, 121, 121, 121, 121, 121, 120, 120, 120, 119, 119, 119, 119, 119, 119, 120, 121, 123, 123, 124, 123, 124, 124, 124, 124, 124, 125, 125, 125, 125, 124, 123, 122, 121, 120, 121, 123, 123, 123, 122, 122, 122, 122, 121, 121, 121, 121, 121, 122, 122, 1

**Calculating 3rd Quartile of HR in each Time Interval**

In [6]:
import numpy as np

print_detailed_output = False

# Initialize the list to store the results
heart_rate_outputs = []

# Loop through each string in the time_intervals list
for interval_str in time_intervals:
    # a. Parse the string to get start and end minutes
    parts = interval_str.split('-')
    start_min = int(parts[0])
    end_min = int(parts[1])

    # b. Convert minutes to a range of seconds
    start_sec = (start_min - 1) * 60 + 1
    end_sec = end_min * 60

    # c. Convert the seconds range to list indices for slicing
    start_index = start_sec - 1
    end_index = end_sec

    # d. Extract the heart rate values for the calculated time window
    if start_index >= len(all_heart_rates):
        print(f"Interval {interval_str} ({start_sec}-{end_sec}s) is completely outside the available data range. Skipping.")
        # Optionally append a placeholder like None or NaN
        heart_rate_outputs.append(None)
        continue

    # Slice the list to get the heart rates in the specified interval
    relevant_heart_rates = all_heart_rates[start_index:end_index]

    if (print_detailed_output):
        print(f"For interval '{interval_str}':")
        print(f"  - Minutes: {start_min}-{end_min}")
        print(f"  - Corresponding Seconds: {start_sec}-{end_sec}")
        print(f"  - Corresponding Indices: {start_index}-{end_index-1}")
        print(f"  - Number of data points found: {len(relevant_heart_rates)}")

    # e. Calculate the third quartile
    if relevant_heart_rates:
        third_quartile = np.quantile(relevant_heart_rates, 0.75)
        heart_rate_outputs.append(round(third_quartile))
    else:
        print("  - No data points found for this interval. Appending None.\n")
        heart_rate_outputs.append(None)



# ***********************

# Manually modifying 5th stage 3rd quartile due to likely HR reading error
heart_rate_outputs[4] = 179

# ***********************



# Final output
print("Third quartile heart rates:")
print(heart_rate_outputs)

Third quartile heart rates:
[135, 145, 156, 171, 179, 187, 193, 197]


**Interactive Lactate Calculation**

In [7]:
# INPUTS ****************

# Testing specs for this step specifically
plot_stages = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
lactate = np.array([0.7, 1.1, 1.0, 2.2, 2.0, 3.5, 6.4, 10.1, 19.2])
confidence = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1]) # 1-10 (scalar for weighting in the best-fit line)

MAX_lactate = 19.2 # <- if there was a max lactate beyond the last stage (went to exhaustion)
MAX_stage = 9

# Threshold guess ranges, threshold must be within this range
AeT_guess_low, AeT_guess_high = 3, 4 
AnT_guess_low, AnT_guess_high = 5, 7

# Fit ranges
AeT_fitrange_low, AeT_fitrange_high = 1, 4
AnT_fitrange_low, AnT_fitrange_high = 4, 10

# ***********************



# Imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar
from ipywidgets import interact, FloatSlider, IntSlider
from IPython.display import display

## Helper Functions

def ridge_regression(x_data, y_data, lambda_val, degree):
    """Perform ridge regression with regularization"""
    X = np.vander(x_data, degree + 1, increasing=True)
    XTX = X.T @ X
    lambda_I = lambda_val * np.eye(degree + 1)
    XTy = X.T @ y_data
    coefficients = np.linalg.solve(XTX + lambda_I, XTy)
    return coefficients

def evaluate_polynomial(coefficients, x):
    """Evaluate polynomial at given x values"""
    return sum(coef * x**i for i, coef in enumerate(coefficients))

def find_closest_root(coefficients, goal):
    """Find x value where polynomial equals goal"""
    def objective(x):
        return abs(evaluate_polynomial(coefficients, x) - goal)
    result = minimize_scalar(objective, bounds=(1, 20), method='bounded')
    return result.x

def calculate_thresholds(AeT_lambda, AeT_degree, AnT_lambda, AnT_degree):
    """Calculate AeT and AnT stages"""
    # Filter data by fit ranges
    AeT_mask = (plot_stages >= AeT_fitrange_low) & (plot_stages <= AeT_fitrange_high)
    AnT_mask = (plot_stages >= AnT_fitrange_low) & (plot_stages <= AnT_fitrange_high)
    
    AeT_stages_filtered = plot_stages[AeT_mask]
    AeT_lactate_filtered = lactate[AeT_mask]
    AeT_confidence_filtered = confidence[AeT_mask]
    
    AnT_stages_filtered = plot_stages[AnT_mask]
    AnT_lactate_filtered = lactate[AnT_mask]
    AnT_confidence_filtered = confidence[AnT_mask]
    
    # Apply confidence weighting
    AeT_stages_confident = np.repeat(AeT_stages_filtered, AeT_confidence_filtered)
    AeT_lactate_confident = np.repeat(AeT_lactate_filtered, AeT_confidence_filtered)
    
    AnT_stages_confident = np.repeat(AnT_stages_filtered, AnT_confidence_filtered)
    AnT_lactate_confident = np.repeat(AnT_lactate_filtered, AnT_confidence_filtered)
    
    # Ridge regression
    AeT_coef = ridge_regression(AeT_stages_confident, AeT_lactate_confident, 
                                10**AeT_lambda, AeT_degree)
    AnT_coef = ridge_regression(AnT_stages_confident, AnT_lactate_confident, 
                                10**AnT_lambda, AnT_degree)
    
    # Calculate AeT (derivative - 0.4 mmol/L per stage = 0)
    AeT_deriv_coef = np.array([AeT_coef[i] * i for i in range(1, len(AeT_coef))])
    AeT_deriv_coef[0] -= 0.4
    
    x_test = np.linspace(AeT_guess_low, AeT_guess_high, 1000)
    derivatives = np.array([abs(evaluate_polynomial(AeT_deriv_coef, x)) for x in x_test])
    AeT_stage = x_test[np.argmin(derivatives)]
    AeT_lactate_val = evaluate_polynomial(AeT_coef, AeT_stage)
    
    # Calculate max stage
    compute_max_stage = find_closest_root(AnT_coef, MAX_lactate)
    
    # Calculate connecting line
    slope = (MAX_lactate - AeT_lactate_val) / (compute_max_stage - AeT_stage)
    intercept = MAX_lactate - (compute_max_stage * slope)
    
    # Calculate AnT (maximum distance between AnT curve and connecting line)
    maximum_stage = compute_max_stage # can switch to MAX_stage for manual input
    x_connect = np.linspace(AeT_stage, maximum_stage, 10000)
    x_connect_filtered = x_connect[(x_connect >= AnT_guess_low) & (x_connect <= AnT_guess_high)]
    
    distances = []
    for x in x_connect_filtered:
        best_fit_y = evaluate_polynomial(AnT_coef, x)
        connecting_y = slope * x + intercept
        dist = abs((connecting_y - best_fit_y) * np.sin(np.arctan(1 / slope)))
        distances.append(dist)
    
    AnT_stage = x_connect_filtered[np.argmax(distances)]
    AnT_lactate_val = evaluate_polynomial(AnT_coef, AnT_stage)
    
    return AeT_stage, AnT_stage, AeT_lactate_val, AnT_lactate_val, AeT_coef, AnT_coef, slope, intercept, compute_max_stage

# Plotting

def update_plot(AeT_lambda=0.0, AeT_degree=3, AnT_lambda=0.0, AnT_degree=3):
    """Update the plot based on slider values"""
    
    AeT_stage, AnT_stage, AeT_lactate_val, AnT_lactate_val, AeT_coef, AnT_coef, slope, intercept, compute_max_stage = \
        calculate_thresholds(AeT_lambda, AeT_degree, AnT_lambda, AnT_degree)
    
    # Generate curves
    x_AeT = np.linspace(AeT_fitrange_low, AeT_fitrange_high, 300)
    y_AeT = [evaluate_polynomial(AeT_coef, x) for x in x_AeT]
    
    x_AnT = np.linspace(AnT_fitrange_low, compute_max_stage, 300)
    y_AnT = [evaluate_polynomial(AnT_coef, x) for x in x_AnT]
    
    x_connect = np.linspace(AeT_stage, compute_max_stage, 300)
    y_connect = slope * x_connect + intercept
    
    # Plotting
    plt.figure(figsize=(12, 7))
    
    # Raw data
    plt.scatter(plot_stages, lactate, color='red', s=80, label='Raw Data', zorder=5)
    
    # Thresholds
    plt.scatter(AeT_stage, AeT_lactate_val, color='orange', s=120, marker='^', 
                label='Aerobic Threshold', zorder=6, edgecolors='black', linewidths=1.5)
    plt.scatter(AnT_stage, AnT_lactate_val, color='orange', s=120, marker='^', 
                label='Anaerobic Threshold', zorder=6, edgecolors='black', linewidths=1.5)
    
    # Curves
    plt.plot(x_AeT, y_AeT, color='blue', linewidth=2.5, label='Aerobic Threshold Curve')
    plt.plot(x_AnT, y_AnT, color='green', linewidth=2.5, label='Anaerobic Threshold Curve')
    plt.plot(x_connect, y_connect, color='black', linewidth=1.5, alpha=0.5, 
             linestyle='--', label='Connecting Line')
    
    plt.xlabel('Stage', fontsize=14, family='monospace')
    plt.ylabel('Lactate (mmol/L)', fontsize=14, family='monospace')
    plt.title('Lactate Threshold Analysis', fontsize=18, family='monospace', weight='bold')
    plt.xlim(0, 12)
    plt.ylim(0, 20)
    plt.grid(True, alpha=0.3)
    plt.legend(loc='upper left', fontsize=10, framealpha=0.9)
    
    plt.tight_layout()
    plt.show()
    
    # Display threshold values
    print(f"{'='*50}")
    print(f"Aerobic Threshold Stage:   {AeT_stage:.3f}")
    print(f"Anaerobic Threshold Stage: {AnT_stage:.3f}")
    print(f"{'='*50}")

# Create interactive plot
interact(update_plot,
         AeT_lambda=FloatSlider(min=-3, max=3, step=0.1, value=0, 
                                description='AeT Regularization:', 
                                style={'description_width': '150px'}),
         AeT_degree=IntSlider(min=2, max=8, step=1, value=3, 
                             description='AeT Polynomial Degree:', 
                             style={'description_width': '150px'}),
         AnT_lambda=FloatSlider(min=-3, max=3, step=0.1, value=0, 
                               description='AnT Regularization:', 
                               style={'description_width': '150px'}),
         AnT_degree=IntSlider(min=2, max=8, step=1, value=3, 
                             description='AnT Polynomial Degree:', 
                             style={'description_width': '150px'}));

interactive(children=(FloatSlider(value=0.0, description='AeT Regularization:', max=3.0, min=-3.0, style=Slide…

**HR Best-fit Line and AeT + AnT HRs**

In [8]:
# INPUTS ****************

AeT_stage = 3.280
AnT_stage = 6.556

# ***********************



import numpy as np
import matplotlib.pyplot as plt

# The degree of the HR polynomial
degree = 3

# Calculate the best-fit polynomial coefficients
try:
    coefficients = np.polyfit(stages, heart_rate_outputs, degree)
except np.linalg.LinAlgError as e:
    print(f"Error fitting polynomial: {e}")
    coefficients = []

if len(coefficients) > 0:
    print("Polynomial Equation for HR:")
    print(f"y = {coefficients[0]:.4f}x^3 + {coefficients[1]:.4f}x^2 + {coefficients[2]:.4f}x + {coefficients[3]:.4f}")

    # Create the polynomial function from the coefficients
    polynomial_function = np.poly1d(coefficients)

    if print_HR_graph:
        # Create a smooth range of x-values for plotting the curve
        x_fit = np.linspace(min(stages), max(stages), 400)

        # Calculate the corresponding y-values using the polynomial function
        y_fit = polynomial_function(x_fit)

        # Set up the plot
        plt.figure(figsize=(10, 6))
        plt.scatter(stages, heart_rate_outputs, color='red', label='Original Data Points')
        plt.plot(x_fit, y_fit, color='blue', linewidth=2, label=f'Third-Degree Best-Fit Polynomial')

        # Add titles and labels
        plt.title('Third-Degree Polynomial Best-Fit Curve', fontsize=16)
        # plt.xlabel('X-axis Data', fontsize=12)
        # plt.ylabel('Y-axis Data', fontsize=12)
        plt.legend()
        plt.grid(True)

        # Display the plot
        plt.show()

if print_all_HR:
  import matplotlib.pyplot as plt

  plt.figure(figsize=(12, 6))
  plt.plot(all_heart_rates)
  plt.title('Heart Rate Over Time')
  plt.xlabel('Time (data points)')
  plt.ylabel('Heart Rate (bpm)')
  plt.grid(True)
  plt.show()


y_for_AeT_stage = polynomial_function(AeT_stage)
y_for_AnT_stage = polynomial_function(AnT_stage)

# Print the results
print(f"\n--- Lactate Threshold HR Values ---")
print(f"AeT stage {AeT_stage} | HR: {y_for_AeT_stage:.4f}")
print(f"AnT stage {AnT_stage} | HR: {y_for_AnT_stage:.4f}")

Polynomial Equation for HR:
y = -0.1187x^3 + 0.9058x^2 + 9.3817x + 124.2857

--- Lactate Threshold HR Values ---
AeT stage 3.28 | HR: 160.6149
AnT stage 6.556 | HR: 191.2821
